<a href="https://colab.research.google.com/github/smf-9000/kaggle/blob/main/%5B004_9%5D_NaN_AE_imputer_tps_sep_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Here I use Deep Autoencoder to impute NaN values.

In [ ]:
%config Completer.use_jedi = False

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow, imread

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, KBinsDiscretizer, MinMaxScaler, QuantileTransformer
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.impute import SimpleImputer


import scipy.stats as stats

import lightgbm as lgb
import warnings

import optuna

import gc

In [ ]:
R_SEED = 37
N_FOLDS = 5

In [ ]:
submission_ex = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/sample_solution.csv')
train_data_b = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/train.csv')
test_data_b  = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/test.csv')

In [ ]:
target_data = train_data_b[['claim']].copy()
for_me_data_train = train_data_b[['id']].copy()
for_me_data_test = test_data_b[['id']].copy()
train_data_b.drop(['id', 'claim'], axis=1, inplace=True)
submit_data = test_data_b[['id']].copy()
test_data_b.drop(['id'], axis=1, inplace=True)
n_missing_train = train_data_b.isna().sum(axis=1)

In [ ]:
all_data = pd.concat([train_data_b, test_data_b])
all_data.reset_index(drop=True, inplace=True)

In [ ]:
all_data_normalized = StandardScaler().fit_transform(all_data)
all_data = pd.DataFrame(all_data_normalized.copy(), columns=all_data.columns)
del all_data_normalized
gc.collect()

In [ ]:
mm_scaler = MinMaxScaler()
all_data_mm = mm_scaler.fit_transform(all_data)
all_data = pd.DataFrame(all_data_mm.copy(), columns=all_data.columns)
del all_data_mm
gc.collect()

In [ ]:
code_data = all_data.isna().astype(int)

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='median') # add_indicator=True, 

old_features = list(all_data.columns)
all_data = pd.DataFrame(imputer.fit_transform(all_data))
features = old_features # + ['ind_for_' + str(e) for e in imputer.indicator_.features_]
all_data.columns = features

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, Model

In [ ]:
def AE(input_shape=all_data.shape[1]):
    
    x_input = layers.Input(shape=input_shape)
    
    X = layers.Dense(128, activation='selu')(x_input)
    X = layers.BatchNormalization()(X)  
    X = layers.Dense(136, activation='selu')(X)
    X = layers.BatchNormalization()(X)  
    X = layers.Dense(144, activation='selu')(X)
    X = layers.BatchNormalization()(X)  
    X = layers.Dense(152, activation='selu')(X)
    X = layers.BatchNormalization()(X)  
    X = layers.Dense(160, activation='selu')(X)
    X = layers.BatchNormalization()(X)  
    X = layers.Dense(152, activation='selu')(X)
    X = layers.BatchNormalization()(X)  
    X = layers.Dense(144, activation='selu')(X)
    X = layers.BatchNormalization()(X)  
    X = layers.Dense(136, activation='selu')(X)
    X = layers.BatchNormalization()(X)  
    X = layers.Dense(128, activation='selu')(X)
    X = layers.BatchNormalization()(X) 
    
    X = layers.Dense(all_data.shape[1], activation='relu')(X)
    
    model = models.Model(inputs = x_input, outputs = X, name='TPS-SEP21_AE')
    return model

In [ ]:
gc.collect()

In [ ]:
del model

In [ ]:
tf.random.set_seed(R_SEED)

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

In [ ]:
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
tf.config.set_soft_device_placement(True)

In [ ]:
gc.collect()

In [ ]:
with tpu_strategy.scope():
    model_1 = AE()

    model_1.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
        loss = 'mse',
        metrics = ['mse'])

    history = model_1.fit(
    all_data,
    all_data,
    epochs = 100,
    shuffle = True,
#     validation_split = 0.2,
    batch_size = 1024,)

In [ ]:
_p = model_1.predict(all_data.values, batch_size=1024)

In [ ]:
tmp = pd.DataFrame(_p, columns=all_data.columns, index=all_data.index)
tmp1 = pd.DataFrame(mm_scaler.inverse_transform(tmp), columns=all_data.columns, index=all_data.index)

In [ ]:
imputer_data_all = pd.DataFrame(code_data.values * tmp1.values, columns=all_data.columns, index=all_data.index)

In [ ]:
imputer_data_all.to_csv('imputer_data_all.csv', index=False)